In [9]:
import csv
import os
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
from openpyxl import Workbook, load_workbook
import numpy as np

pd.set_option('display.max_colwidth', None)


In [10]:
# path to the html files
directory = Path('../scraping-script/Full_HTML_Data/HTML')

if directory.is_dir():
    print("oui")
else:
    print("No")

oui


In [11]:
f360_data_set_csv = "fusion_360_api_data.csv"

In [12]:
#open csv file for writing
with open(f360_data_set_csv, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # write the header row
    writer.writerow(['Name', 'Type', 'Parent Object', 'Description'])

    # iterating over each file in directory
    for filename in os.listdir(directory):
        if filename.endswith('.htm') or filename.endswith('.html'):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                soup = BeautifulSoup(file, 'html.parser')
              # extracting the name and type from the h1 tag
                name_type = soup.h1.text.strip() if soup.h1 else 'N/A'
                name, type_ = name_type.rsplit(' ', 1) if ' ' in name_type else (name_type, 'N/A')
                # extracting the parent object
                parent_object_tag = soup.find('p', class_='api')
                parent_object = parent_object_tag.a.text.strip() if parent_object_tag and parent_object_tag.a else 'N/A'
                # extract the description
                description_tag = parent_object_tag.find_next_sibling('p', class_='api') if parent_object_tag else None
                description = description_tag.text.strip() if description_tag else 'N/A'
                # write to CSV
                writer.writerow([name, type_, parent_object, description])

In [13]:
f360_full_df = pd.read_csv(f360_data_set_csv)

f360_full_df

,Name,Type,Parent Object,Description
0,Analyze Interference API Sample API,Sample,NaN,NaN
1,AngleExtentDefinition,Object,NaN,NaN
2,AngleExtentDefinition.angle,Property,AngleExtentDefinition,"Gets the ModelParameter that defines the angle. The value of the angle can be edited by using the properties on the ModelParameter object to edit the parameter.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
3,AngleExtentDefinition.classType,Method,AngleExtentDefinition,"Static function that all classes support that returns the type of the class as a string. The returned string matches the string returned by the objectType property. For example if you have a reference to an object and you want to check if it's a SketchLine you can use myObject.objectType == fusion.SketchLine.classType().Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
4,AngleExtentDefinition.isSymmetric,Property,AngleExtentDefinition,"Gets and sets if the angle extent is in one direction or symmetric. For a hole this property will always return false and setting it is ignored.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
...,...,...,...,...
8038,Workspace.toolClipFilename,Property,Workspace,"Gets or sets the full filename of the image file (png) used for the tool clip. the tool clip is the image shown when the user hovers the mouse over the workspace name in the workspace drop-down.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8039,Workspace.tooltip,Property,Workspace,"Gets or sets the tooltip text displayed for the workspace. This is the first line of text shown when the user hovers over the workspace name in the Fusion 360 toolbar drop-down. This is typically the name of the workspace. This is different from the name in the that the name is a short name shown in the drop-down. The tooltip is only shown when the user hovers over the name and box appears providing more information about the workspace. For example, the name of the model workspace is ""Model"" and the tooltip is ""Model Workspace"".Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8040,Workspace.tooltipDescription,Property,Workspace,"Gets or sets the tooltip description displayed for the workspace. The tooltip description is a longer description of the workspace and is only displayed when the user hovers over the workspace name in the Fusion 360 toolbar drop-down. The pop-up dialog that appears contains the tooltip, the tooltip description, and the toolclip which is a picture.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8041,Write user interface to a file API,Sample,NaN,NaN


In [14]:
f360_df_filtered = f360_full_df[f360_full_df["Type"] != "Sample"]

f360_df_filtered

,Name,Type,Parent Object,Description
1,AngleExtentDefinition,Object,NaN,NaN
2,AngleExtentDefinition.angle,Property,AngleExtentDefinition,"Gets the ModelParameter that defines the angle. The value of the angle can be edited by using the properties on the ModelParameter object to edit the parameter.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
3,AngleExtentDefinition.classType,Method,AngleExtentDefinition,"Static function that all classes support that returns the type of the class as a string. The returned string matches the string returned by the objectType property. For example if you have a reference to an object and you want to check if it's a SketchLine you can use myObject.objectType == fusion.SketchLine.classType().Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
4,AngleExtentDefinition.isSymmetric,Property,AngleExtentDefinition,"Gets and sets if the angle extent is in one direction or symmetric. For a hole this property will always return false and setting it is ignored.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
5,AngleExtentDefinition.isValid,Property,AngleExtentDefinition,"Indicates if this object is still valid, i.e. hasn't been deleted or some other action done to invalidate the reference.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
...,...,...,...,...
8037,Workspace.toolbarPanels,Property,Workspace,"Gets the collection containing the panels associated with this workspace. It's through this collection that you can add new toolbar panels.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8038,Workspace.toolClipFilename,Property,Workspace,"Gets or sets the full filename of the image file (png) used for the tool clip. the tool clip is the image shown when the user hovers the mouse over the workspace name in the workspace drop-down.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8039,Workspace.tooltip,Property,Workspace,"Gets or sets the tooltip text displayed for the workspace. This is the first line of text shown when the user hovers over the workspace name in the Fusion 360 toolbar drop-down. This is typically the name of the workspace. This is different from the name in the that the name is a short name shown in the drop-down. The tooltip is only shown when the user hovers over the name and box appears providing more information about the workspace. For example, the name of the model workspace is ""Model"" and the tooltip is ""Model Workspace"".Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8040,Workspace.tooltipDescription,Property,Workspace,"Gets or sets the tooltip description displayed for the workspace. The tooltip description is a longer description of the workspace and is only displayed when the user hovers over the workspace name in the Fusion 360 toolbar drop-down. The pop-up dialog that appears contains the tooltip, the tooltip description, and the toolclip which is a picture.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."


In [15]:
unique_types = f360_df_filtered["Type"].unique()

unique_types

array(['Object', 'Property', 'Method', 'Enumerator', 'Event', nan,
       'Samples', 'Filters', 'Manual', 'GitHub', 'Interface)', 'New'],
      dtype=object)

In [16]:
outliers_df = f360_df_filtered.loc[(f360_df_filtered["Type"] == "GitHub") | (f360_df_filtered["Type"] == "Samples") | (f360_df_filtered["Type"] == "Manual") | (f360_df_filtered["Type"] == "Filters")
                             | (f360_df_filtered["Type"] == "Interface")]



outliers_df.head()

f360_df_filtered = f360_df_filtered.drop(outliers_df.index)

f360_df_filtered

,Name,Type,Parent Object,Description
1,AngleExtentDefinition,Object,NaN,NaN
2,AngleExtentDefinition.angle,Property,AngleExtentDefinition,"Gets the ModelParameter that defines the angle. The value of the angle can be edited by using the properties on the ModelParameter object to edit the parameter.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
3,AngleExtentDefinition.classType,Method,AngleExtentDefinition,"Static function that all classes support that returns the type of the class as a string. The returned string matches the string returned by the objectType property. For example if you have a reference to an object and you want to check if it's a SketchLine you can use myObject.objectType == fusion.SketchLine.classType().Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
4,AngleExtentDefinition.isSymmetric,Property,AngleExtentDefinition,"Gets and sets if the angle extent is in one direction or symmetric. For a hole this property will always return false and setting it is ignored.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
5,AngleExtentDefinition.isValid,Property,AngleExtentDefinition,"Indicates if this object is still valid, i.e. hasn't been deleted or some other action done to invalidate the reference.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
...,...,...,...,...
8037,Workspace.toolbarPanels,Property,Workspace,"Gets the collection containing the panels associated with this workspace. It's through this collection that you can add new toolbar panels.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8038,Workspace.toolClipFilename,Property,Workspace,"Gets or sets the full filename of the image file (png) used for the tool clip. the tool clip is the image shown when the user hovers the mouse over the workspace name in the workspace drop-down.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8039,Workspace.tooltip,Property,Workspace,"Gets or sets the tooltip text displayed for the workspace. This is the first line of text shown when the user hovers over the workspace name in the Fusion 360 toolbar drop-down. This is typically the name of the workspace. This is different from the name in the that the name is a short name shown in the drop-down. The tooltip is only shown when the user hovers over the name and box appears providing more information about the workspace. For example, the name of the model workspace is ""Model"" and the tooltip is ""Model Workspace"".Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8040,Workspace.tooltipDescription,Property,Workspace,"Gets or sets the tooltip description displayed for the workspace. The tooltip description is a longer description of the workspace and is only displayed when the user hovers over the workspace name in the Fusion 360 toolbar drop-down. The pop-up dialog that appears contains the tooltip, the tooltip description, and the toolclip which is a picture.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."


In [17]:
unique_types2 = f360_df_filtered["Type"].unique()

null_names = f360_df_filtered["Name"].isnull()

null_description = f360_df_filtered["Description"].isnull()

null_parent_object = f360_df_filtered["Parent Object"].isnull()

null_types = f360_df_filtered["Type"].isnull()


In [18]:

def null_names_rows(y):
    x = y[null_names]
    return x


def null_type_rows(y):
    x = y[null_types]
    return x


def null_description_rows(y):
    x = y[null_description]
    return x

def null_parent_object_rows(y):
    x = y[null_parent_object]
    return x
    
null_names_rows(f360_df_filtered)

,Name,Type,Parent Object,Description
295,NaN,NaN,NaN,NaN
379,NaN,NaN,NaN,NaN
658,NaN,NaN,NaN,NaN
850,NaN,NaN,NaN,NaN
1307,NaN,NaN,NaN,NaN
1322,NaN,NaN,NaN,NaN
1369,NaN,NaN,NaN,NaN
1831,NaN,NaN,NaN,NaN
2158,NaN,NaN,NaN,NaN
2448,NaN,NaN,NaN,NaN


In [19]:

i4711 = f360_df_filtered.loc[4711]

i4711

Name                                                                                                                                                                                                                                                                                                                                                                                                                                                                        NaN
Type                                                                                                                                                                                                                                                                                                                                                                                                                                                                        NaN
Parent Object                                                           

In [20]:
f360_df_filtered.loc[4711, "Name"] = "~Information on Pallets"

f360_df_filtered.loc[4711, "Type"] = "~Miscellaneous"


#i4711 = f360_df_filtered.loc[4711]
#i4711
i4711

Name                                                                                                                                                                                                                                                                                                                                                                                                                                                    ~Information on Pallets
Type                                                                                                                                                                                                                                                                                                                                                                                                                                                             ~Miscellaneous
Parent Object                                                           

In [21]:
#empty_rows =  f360_df_filtered[null_names] and f360_df_filtered[null_description] and f360_df_filtered[null_parent_object] and f360_df_filtered[null_types]
#empty_rows

f360_df_filtered.dropna(axis = 0, how = 'all', inplace = True)

null_names_rows(f360_df_filtered)

C:\Users\ellio\AppData\Local\Temp\ipykernel_36156\26298854.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = y[null_names]


,Name,Type,Parent Object,Description
4711,~Information on Pallets,~Miscellaneous,NaN,"Palettes are very distinct from command dialogs in several ways. First, palettes are not associated with a command and do not have the same lifetime rules that a command has. A palette can be displayed while the user runs commands and interacts with Fusion 360. A palette is not even associated with a particular document. When a palette is displayed it will remain displayed for the entire Fusion 360 session unless you or the user closes it."
7380,NaN,NaN,NaN,"If you think of a thread as a sequence of instructions that are processed by the computer, multi-threading allows many threads to be processed simultaneously. Besides being used to speed up the processing of complex tasks, multiple threads are also very useful in interactive applications when you need to have background processing occurring but still want to allow the user to interact with the application. That's the big benefit of what's provided by the Fusion 360 API, as you'll see in the example below."
8042,NaN,NaN,NaN,"Now that you’ve seen the basic process of creating and debugging a script, here is some more information about the details of both scripts and add-ins."


In [22]:
f360_df_filtered = f360_df_filtered = f360_df_filtered.drop(7380, axis="index")

f360_df_filtered = f360_df_filtered = f360_df_filtered.drop(8042, axis="index")


In [23]:
null_names_rows(f360_df_filtered)

C:\Users\ellio\AppData\Local\Temp\ipykernel_36156\26298854.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = y[null_names]


,Name,Type,Parent Object,Description
4711,~Information on Pallets,~Miscellaneous,NaN,"Palettes are very distinct from command dialogs in several ways. First, palettes are not associated with a command and do not have the same lifetime rules that a command has. A palette can be displayed while the user runs commands and interacts with Fusion 360. A palette is not even associated with a particular document. When a palette is displayed it will remain displayed for the entire Fusion 360 session unless you or the user closes it."


In [24]:
f360_df_filtered

,Name,Type,Parent Object,Description
1,AngleExtentDefinition,Object,NaN,NaN
2,AngleExtentDefinition.angle,Property,AngleExtentDefinition,"Gets the ModelParameter that defines the angle. The value of the angle can be edited by using the properties on the ModelParameter object to edit the parameter.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
3,AngleExtentDefinition.classType,Method,AngleExtentDefinition,"Static function that all classes support that returns the type of the class as a string. The returned string matches the string returned by the objectType property. For example if you have a reference to an object and you want to check if it's a SketchLine you can use myObject.objectType == fusion.SketchLine.classType().Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
4,AngleExtentDefinition.isSymmetric,Property,AngleExtentDefinition,"Gets and sets if the angle extent is in one direction or symmetric. For a hole this property will always return false and setting it is ignored.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
5,AngleExtentDefinition.isValid,Property,AngleExtentDefinition,"Indicates if this object is still valid, i.e. hasn't been deleted or some other action done to invalidate the reference.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
...,...,...,...,...
8036,Workspace.resourceFolder,Property,Workspace,"Gets or sets the resource folder.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8037,Workspace.toolbarPanels,Property,Workspace,"Gets the collection containing the panels associated with this workspace. It's through this collection that you can add new toolbar panels.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8038,Workspace.toolClipFilename,Property,Workspace,"Gets or sets the full filename of the image file (png) used for the tool clip. the tool clip is the image shown when the user hovers the mouse over the workspace name in the workspace drop-down.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8039,Workspace.tooltip,Property,Workspace,"Gets or sets the tooltip text displayed for the workspace. This is the first line of text shown when the user hovers over the workspace name in the Fusion 360 toolbar drop-down. This is typically the name of the workspace. This is different from the name in the that the name is a short name shown in the drop-down. The tooltip is only shown when the user hovers over the name and box appears providing more information about the workspace. For example, the name of the model workspace is ""Model"" and the tooltip is ""Model Workspace"".Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."


In [25]:
null_type_rows(f360_df_filtered)

C:\Users\ellio\AppData\Local\Temp\ipykernel_36156\26298854.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = y[null_types]


,Name,Type,Parent Object,Description
4711,~Information on Pallets,~Miscellaneous,NaN,"Palettes are very distinct from command dialogs in several ways. First, palettes are not associated with a command and do not have the same lifetime rules that a command has. A palette can be displayed while the user runs commands and interacts with Fusion 360. A palette is not even associated with a particular document. When a palette is displayed it will remain displayed for the entire Fusion 360 session unless you or the user closes it."


In [26]:
null_description_rows(f360_df_filtered)

C:\Users\ellio\AppData\Local\Temp\ipykernel_36156\26298854.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = y[null_description]


,Name,Type,Parent Object,Description
1,AngleExtentDefinition,Object,NaN,NaN
8,AngleValueCommandInput,Object,NaN,NaN
36,Appearance,Object,NaN,NaN
37,Appearances,Object,NaN,NaN
38,AppearanceSourceTypes,Enumerator,NaN,NaN
...,...,...,...,...
7995,WorkspaceEvent,Object,NaN,NaN
7996,WorkspaceEventArgs,Object,NaN,NaN
8002,WorkspaceEventHandler,Object,NaN,NaN
8011,WorkspaceList,Object,NaN,NaN


In [27]:
null_parent_object_rows(f360_df_filtered)

C:\Users\ellio\AppData\Local\Temp\ipykernel_36156\26298854.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = y[null_parent_object]


,Name,Type,Parent Object,Description
1,AngleExtentDefinition,Object,NaN,NaN
8,AngleValueCommandInput,Object,NaN,NaN
36,Appearance,Object,NaN,NaN
37,Appearances,Object,NaN,NaN
38,AppearanceSourceTypes,Enumerator,NaN,NaN
...,...,...,...,...
7995,WorkspaceEvent,Object,NaN,NaN
7996,WorkspaceEventArgs,Object,NaN,NaN
8002,WorkspaceEventHandler,Object,NaN,NaN
8011,WorkspaceList,Object,NaN,NaN


In [28]:
outliers_parent_object = f360_df_filtered[(f360_df_filtered['Parent Object'].isna()) & (f360_df_filtered['Description'].notna())]
outliers_parent_object


,Name,Type,Parent Object,Description
1203,Command,Object,NaN,"The file referenced must be a local file and cannot be a url. However, you can use a local html file that redirects to a url."
4711,~Information on Pallets,~Miscellaneous,NaN,"Palettes are very distinct from command dialogs in several ways. First, palettes are not associated with a command and do not have the same lifetime rules that a command has. A palette can be displayed while the user runs commands and interacts with Fusion 360. A palette is not even associated with a particular document. When a palette is displayed it will remain displayed for the entire Fusion 360 session unless you or the user closes it."


In [29]:
f360_df_filtered = f360_df_filtered.drop(1203, axis="index")

In [30]:
f360_df_filtered

,Name,Type,Parent Object,Description
1,AngleExtentDefinition,Object,NaN,NaN
2,AngleExtentDefinition.angle,Property,AngleExtentDefinition,"Gets the ModelParameter that defines the angle. The value of the angle can be edited by using the properties on the ModelParameter object to edit the parameter.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
3,AngleExtentDefinition.classType,Method,AngleExtentDefinition,"Static function that all classes support that returns the type of the class as a string. The returned string matches the string returned by the objectType property. For example if you have a reference to an object and you want to check if it's a SketchLine you can use myObject.objectType == fusion.SketchLine.classType().Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
4,AngleExtentDefinition.isSymmetric,Property,AngleExtentDefinition,"Gets and sets if the angle extent is in one direction or symmetric. For a hole this property will always return false and setting it is ignored.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
5,AngleExtentDefinition.isValid,Property,AngleExtentDefinition,"Indicates if this object is still valid, i.e. hasn't been deleted or some other action done to invalidate the reference.Defined in namespace ""adsk.fusion"" and the header file is ""adsk/fusion/Features/AngleExtentDefinition.h""."
...,...,...,...,...
8036,Workspace.resourceFolder,Property,Workspace,"Gets or sets the resource folder.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8037,Workspace.toolbarPanels,Property,Workspace,"Gets the collection containing the panels associated with this workspace. It's through this collection that you can add new toolbar panels.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8038,Workspace.toolClipFilename,Property,Workspace,"Gets or sets the full filename of the image file (png) used for the tool clip. the tool clip is the image shown when the user hovers the mouse over the workspace name in the workspace drop-down.Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."
8039,Workspace.tooltip,Property,Workspace,"Gets or sets the tooltip text displayed for the workspace. This is the first line of text shown when the user hovers over the workspace name in the Fusion 360 toolbar drop-down. This is typically the name of the workspace. This is different from the name in the that the name is a short name shown in the drop-down. The tooltip is only shown when the user hovers over the name and box appears providing more information about the workspace. For example, the name of the model workspace is ""Model"" and the tooltip is ""Model Workspace"".Defined in namespace ""adsk.core"" and the header file is ""adsk/core/UserInterface/Workspace.h""."


In [31]:
f360_df_filtered.to_csv("fusion_360_api_data_cleaned.csv", index=False)

In [32]:
#f360_df_filtered.to_csv("../scraping-script/Filtered_F360_CSV.csv")
#f360_df_filtered.to_csv("../scraping-script/Filtered_F360_exl.pyxl")